In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
import random
random.seed(42)
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(42)
import gc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from glob import glob
import os
import sys
import tensorflow as tf
import tensorflow_data_validation as tfdv

import time
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import QuantileTransformer
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
import logging
import logging.handlers

In [3]:
print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 0.30.0


### Preprocessing

### train / val 분리

In [4]:
path = './dataset_/energy/energy'
train = pd.read_csv(path+'/train.csv', encoding = 'cp949')
test = pd.read_csv(path+'/test.csv', encoding = 'cp949')
sample_submission = pd.read_csv(path + '/sample_submission.csv')

train.columns = ['num','date_time','energy','temp','wind_spd','humid','Precip','sun','non_elec','sunlight']
# train.to_csv('train_.csv',encoding='cp949')
test.columns = ['num','date_time','temp','wind_spd','humid','Precip','sun','non_elec','sunlight']
# test.to_csv('test_.csv',encoding='cp949')

In [5]:
import torch
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
# def create_sequences(dataset, start, end, time_steps=24):
#     dataset.set_index('date_time',inplace=True)
    
#     if 'energy' in dataset:    
#         data_set = dataset.iloc[int(start*len(dataset)) : int(end*len(dataset)), :]

#         y_set = dataset[['energy']]
#         x_set = dataset.drop('energy', axis=1)

#         x_ = []
#         y_ = []
#         for i in range(len(y_set) - time_steps + 1):
#             x_.append(x_set.iloc[i : (i + time_steps),:])
#             y_.append(y_set.iloc[i : (i + time_steps),:])
            
#         return np.stack(x_), np.stack(y_)
        
#     # test_set 에서 energy가 없는 경우
#     else:
#         data_set = dataset.iloc[int(start*len(dataset)) : int(end*len(dataset)), :]
#         for i in range(len(y_set) - time_steps + 1):
#             x_.append(x_set.iloc[i : (i + time_steps),:])

#         return np.stack(x_), _


# # dataset
# def get_each_id_set(dataset, start, end, steps=24):
#     data_dic = {}

#     x_lst = []
#     y_lst = []
#     for i in range(1,61):
#         x_, y_ = create_sequences(dataset[dataset.num==i], start, end, steps)
#         x_lst.append(x_)
#         y_lst.append(y_)

#     return x_lst, y_lst

# class CustomDataset(Dataset):    
#     def __init__(self, 
#                  dataset,
#                  start = 0,
#                  end = 1,
#                  steps = 24
#                  ):
        
#         self.dataset = dataset
#         self.steps = steps
#         self.start = start
#         self.end = end

#         self.x, self.y = get_each_id_set(dataset, start, end, steps)
    
    
#     def __len__(self):
#         return len(self.x)
    
    
#     def __getitem__(self, idx):
        
#         return self.x[idx], self.y[idx]

In [7]:
train

,num,date_time,energy,temp,wind_spd,humid,Precip,sun,non_elec,sunlight
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [8]:
def create_sequences(dataset, start, end):
    dataset.set_index('date_time',inplace=True)
    data_set = dataset.iloc[int(start*len(dataset)) : int(end*len(dataset)), :]
    
    return data_set 

# dataset
def get_each_id_set(dataset, num, start, end):
    data_dic = {}

    x_lst = 0
    for i in range(1,num+1):
        x_ = create_sequences(dataset[dataset.num==i], start, end)
        
        if type(x_lst) == type(0):
            x_lst = x_
            
        else:
            x_lst = pd.concat([x_lst, x_])

            
    return x_lst.reset_index(drop=True)


In [28]:
train_ = get_each_id_set(train, 60, 0, 0.8)
val_ = get_each_id_set(train, 60, 0.8, 1)

In [29]:
def get_group(dataset):
    group = dataset[['num', 'energy', 'temp', 'wind_spd', 'humid', 'Precip', 'sun', 'non_elec', 'sunlight']].groupby('num').apply(lambda r: (
                r['num'].values,
                r['energy'].values,
                r['temp'].values,
                r['wind_spd'].values,
                r['humid'].values,
                r['Precip'].values,
                r['sun'].values,
                r['non_elec'].values,
                r['sunlight'].values))
    return group

In [30]:
train_group = get_group(train_)
val_group = get_group(val_)

In [31]:
# def create_sequences(dataset, batch_size, start, end, time_steps=24):
#     dataset.set_index('date_time',inplace=True)
#     data_set = dataset.iloc[int(start*len(dataset)) : int(end*len(dataset)), :]

#     num = []
#     energy = []
#     temp = []
#     wind_spd = []
#     humid = []
#     precip = []
#     sun = []
#     non_elec = []
#     sunlight = []

#     if 'energy' in data_set:
#         for i in range(len(y_set) - time_steps + 1):
#             num.append(data_set.num[i : (i + time_steps)].tolist())
#             energy.append(data_set.energy[i : (i + time_steps)].tolist())
#             temp.append(data_set.temp[i : (i + time_steps)].tolist())
#             wind_spd.append(data_set.wind_spd[i : (i + time_steps)].tolist())
#             humid.append(data_set.humid[i : (i + time_steps)].tolist())
#             precip.append(data_set.precip[i : (i + time_steps)].tolist())
#             sun.append(data_set.sun[i : (i + time_steps)].tolist())
#             non_elec.append(data_set.non_elec[i : (i + time_steps)].tolist())
#             sunlight.append(data_set.sunlight[i : (i + time_steps)].tolist())

#     else:
#         for i in range(len(y_set) - time_steps + 1):
#             num.append(data_set.num[i : (i + time_steps)].tolist())
#     #         energy.append(data_set.energy[i : (i + time_steps)].tolist())
#             temp.append(data_set.temp[i : (i + time_steps)].tolist())
#             wind_spd.append(data_set.wind_spd[i : (i + time_steps)].tolist())
#             humid.append(data_set.humid[i : (i + time_steps)].tolist())
#             precip.append(data_set.precip[i : (i + time_steps)].tolist())
#             sun.append(data_set.sun[i : (i + time_steps)].tolist())
#             non_elec.append(data_set.non_elec[i : (i + time_steps)].tolist())
#             sunlight.append(data_set.sunlight[i : (i + time_steps)].tolist())
        
#         return 
        
#     # test_set 에서 energy가 없는 경우
#     else:
#         data_set = dataset.iloc[int(start*len(dataset)) : int(end*len(dataset)), :]
        
#         for i in range(len(y_set) - time_steps + 1):
#             x_.append(x_set.iloc[i : (i + time_steps),:])

#         return x_, _


# # dataset
# def get_each_id_set(dataset, batch_size, start, end, steps=24):
#     data_dic = {}

#     x_lst = []
#     y_lst = []
#     for i in range(1,61):
#         x_, y_ = create_sequences(dataset[dataset.num==i], batch_size, start, end, steps)
#         x_lst.extend(x_)
#         y_lst.extend(y_)

#     return np.stack(x_lst), np.stack(y_lst)

# class CustomDataset(Dataset):    
#     def __init__(self, 
#                  dataset,
#                  batch_size,
#                  start = 0,
#                  end = 1,
#                  steps = 24
#                  ):
        
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.steps = steps
#         self.start = start
#         self.end = end

#         self.x, self.y = get_each_id_set(dataset, batch_size, start, end, steps)
    
    
#     def __len__(self):
#         return len(self.x)
    
    
#     def __getitem__(self, idx):
        
#         return self.x[idx], self.y[idx]

In [32]:
class Dacon_Dataset(Dataset):
    def __init__(self, dataset, time_steps = 24, test = False):
        self.dataset = dataset
        self.time_steps = time_steps
        self.test = test
        
        self.samples = {}
        self.user_ids = []
        
        for user_id in dataset.index:
            if self.test == True:
                num, temp, wind_spd, humid, precip, sun, non_elec, sunlight = dataset[user_id]

                if len(num) > self.time_steps:
                    total_steps = len(num)

                    for seq in range(len(num) - self.time_steps + 1):
                        self.user_ids.append(f"{user_id}_{seq}")

                        start = seq
                        end = seq + self.time_steps

                        self.samples[f"{user_id}_{seq}"] = (num[start:end], 
#                                                             energy[start:end], 
                                                            temp[start:end], 
                                                            wind_spd[start:end], 
                                                            humid[start:end], 
                                                            precip[start:end],
                                                            sun[start:end],
                                                            non_elec[start:end],
                                                            sunlight[start:end])


            else:
                num, energy, temp, wind_spd, humid, precip, sun, non_elec, sunlight = dataset[user_id]

                if len(num) > self.time_steps:
                    total_steps = len(num)

                    for seq in range(len(num) - self.time_steps + 1):
                        self.user_ids.append(f"{user_id}_{seq}")

                        start = seq
                        end = seq + self.time_steps

                        self.samples[f"{user_id}_{seq}"] = (num[start:end], 
                                                            energy[start:end], 
                                                            temp[start:end], 
                                                            wind_spd[start:end], 
                                                            humid[start:end], 
                                                            precip[start:end],
                                                            sun[start:end],
                                                            non_elec[start:end],
                                                            sunlight[start:end])


                else:
                    user_id = str(user_id)
                    self.user_ids.append(user_id)
                    self.samples[user_id] = (num, 
                                             energy, 
                                             temp, 
                                             wind_spd, 
                                             humid, 
                                             precip, 
                                             sun, 
                                             non_elec, 
                                             sunlight)


    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        if self.test == True:
            
            num_, temp_, wind_spd_, humid_, precip_, sun_, non_elec_, sunlight_ = self.samples[user_id]
            seq_len = len(num_)

            ## for zero padding
            num_ = num_ + 1
#             energy_ = energy_*1000 
            temp_ = temp_*10 + 1
            wind_spd_ = wind_spd_*10 + 1
            humid_ = humid_*10 + 1
            precip_ = precip_* 10 + 1
            sun_ = sun_ * 10 + 1
            non_elec_ = non_elec_ + 1
            sunlight_ = sunlight_ + 1 

            num = np.zeros(self.time_steps, dtype=int)
#             energy = np.zeros(self.time_steps, dtype=float)
            temp = np.zeros(self.time_steps, dtype=int)
            wind_spd = np.zeros(self.time_steps, dtype=int)
            humid = np.zeros(self.time_steps, dtype=int)
            precip = np.zeros(self.time_steps, dtype=int)
            sun = np.zeros(self.time_steps, dtype=int)
            non_elec = np.zeros(self.time_steps, dtype=int)
            sunlight = np.zeros(self.time_steps, dtype=int)

            num[:] = num_
#             energy[:] = energy_
            temp[:] = temp_
            wind_spd[:] = wind_spd_
            humid[:] = humid_
            precip[:] = precip_
            sun[:] = sun_
            non_elec[:] = non_elec_
            sunlight[:] = sunlight_

            num = num[1:]
#             energy = energy[1:]
            temp = temp[1:]
            wind_spd = wind_spd[1:]
            humid = humid[1:]
            precip = precip[1:]
            sun = sun[1:]
            non_elec = non_elec[1:]
            sunlight = sunlight[1:]

            return num, temp, wind_spd, humid, precip, sun, non_elec, sunlight
        
        
        else:
            num_, energy_, temp_, wind_spd_, humid_, precip_, sun_, non_elec_, sunlight_ = self.samples[user_id]
            seq_len = len(num_)

            ## for zero padding

            energy_ = energy_
            temp_ = temp_*10 +1
            wind_spd_ = wind_spd_*10 +1
            humid_ = humid_*10 + 1
            precip_ = precip_* 10 + 1
            sun_ = sun_ * 10 + 1
            non_elec_ = non_elec_ + 1
            sunlight_ = sunlight_ + 1 

            num = np.zeros(self.time_steps, dtype=int)
            energy = np.zeros(self.time_steps, dtype=float)
            temp = np.zeros(self.time_steps, dtype=int)
            wind_spd = np.zeros(self.time_steps, dtype=int)
            humid = np.zeros(self.time_steps, dtype=int)
            precip = np.zeros(self.time_steps, dtype=int)
            sun = np.zeros(self.time_steps, dtype=int)
            non_elec = np.zeros(self.time_steps, dtype=int)
            sunlight = np.zeros(self.time_steps, dtype=int)

            num[:] = num_
            energy[:] = energy_
            temp[:] = temp_
            wind_spd[:] = wind_spd_
            humid[:] = humid_
            precip[:] = precip_
            sun[:] = sun_
            non_elec[:] = non_elec_
            sunlight[:] = sunlight_

            num = num[1:]
            energy = energy[1:]
            temp = temp[1:]
            wind_spd = wind_spd[1:]
            humid = humid[1:]
            precip = precip[1:]
            sun = sun[1:]
            non_elec = non_elec[1:]
            sunlight = sunlight[1:]

            return num, energy, temp, wind_spd, humid, precip, sun, non_elec, sunlight
        

In [33]:
train_set = Dacon_Dataset(train_group,24,False)
train_loader = DataLoader(train_set, batch_size=32, shuffle=False)

val_set = Dacon_Dataset(val_group,24,False)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

### Model

In [34]:
class FFN(nn.Module):
    def __init__(self, state_size=200, dropout=0.2):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)


def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class Energy_Model(nn.Module):
    def __init__(self, 
                 preprocessed_dataset, 
                 time_steps=24, 
                 n_head = 8, 
                 embed_dim=128,
                 num_encoding_layers=6,
                 num_decoding_layers=6,
                 dropout=0.2):
        super(Energy_Model, self).__init__()
        
        self.preprocessed_dataset = preprocessed_dataset
        self.time_steps = time_steps
        self.n_head = n_head
        self.embed_dim = embed_dim
        self.num_encoding_layers = num_encoding_layers
        self.num_decoding_layers = num_decoding_layers
        self.dropout = dropout

        
        
        self.pos_embedding= nn.Embedding(self.time_steps+1, self.embed_dim) ## position
        
        
        self.num_embedding = nn.Embedding(max(self.preprocessed_dataset['num'])+1, self.embed_dim) ## 건물(?)번호
        self.temp_embedding = nn.Embedding(max(self.preprocessed_dataset['temp'])+1, self.embed_dim) ## 온도
        self.wind_spd_embedding = nn.Embedding(max(self.preprocessed_dataset['wind_spd'])+1, self.embed_dim) ## 풍속
        self.humid_embedding = nn.Embedding(max(self.preprocessed_dataset['humid'])+1, self.embed_dim) ## 습도
        self.precip_embedding = nn.Embedding(max(self.preprocessed_dataset['Precip'])+1, self.embed_dim) ## 강수량
        self.sun_embedding = nn.Embedding(max(self.preprocessed_dataset['sun'])+1, self.embed_dim) ## 일조
        self.non_elec_embedding = nn.Embedding(2+1, self.embed_dim) ## 비전기식 유/무
        self.sunlight_embedding = nn.Embedding(2+1, self.embed_dim) ## 태양광 유/무

        
        self.transformer = nn.Transformer(nhead=self.n_head, d_model = self.embed_dim, num_encoder_layers= self.num_encoding_layers, num_decoder_layers= self.num_decoding_layers, dropout = self.dropout)

        self.dropout = nn.Dropout(self.dropout)
        self.layer_normal = nn.LayerNorm(self.embed_dim) 
        self.ffn = FFN(self.embed_dim)
        self.pred = nn.Linear(self.embed_dim, 1)

    def forward(self, num_, temp_, wind_spd_, humid_, precip_, sun_, non_elec_, sunlight_):

        device = num_.device  

        ## embedding layer
        pos_id = torch.arange(self.time_steps).unsqueeze(0).to(device)
        pos_id = self.pos_embedding(pos_id)
        
        num = self.num_embedding(num_)
        temp = self.temp_embedding(temp_)
        wind_spd = self.wind_spd_embedding(wind_spd_)
        humid = self.humid_embedding(humid_)
        precip = self.humid_embedding(precip_)
        sun = self.humid_embedding(sun_)
        non_elec = self.humid_embedding(non_elec_)
        sunlight = self.humid_embedding(sunlight_)

        
        enc = num + temp + wind_spd + humid + precip + sun + non_elec + sunlight
        dec = num + temp + wind_spd + humid + precip + sun + non_elec + sunlight

        enc = enc.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        dec = dec.permute(1, 0, 2)
        mask = future_mask(enc.size(0)).to(device)

        att_output = self.transformer(enc, dec, src_mask=mask, tgt_mask=mask, memory_mask = mask)
        att_output = self.layer_normal(att_output)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

In [35]:
preprocessed_dataset = {}
preprocessed_dataset['num'] = (train.num).astype('int32')
preprocessed_dataset['temp'] = (train.temp*10+1).astype('int32')
preprocessed_dataset['wind_spd'] = (train.wind_spd*10+1).astype('int32')
preprocessed_dataset['humid'] = (train.humid*10+1).astype('int32')
preprocessed_dataset['Precip'] = (train.Precip*10+1).astype('int32')
preprocessed_dataset['sun'] = (train.sun*10+1).astype('int32')
preprocessed_dataset['non_elec'] = (train.non_elec +1).astype('int32')
preprocessed_dataset['sunlight'] = (train.sunlight +1).astype('int32')

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

model = Energy_Model(preprocessed_dataset, time_steps=24, n_head=8, embed_dim=256, num_encoding_layers=6,num_decoding_layers=6, dropout=0.2)


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
criterion = nn.MSELoss()

model.to(device)
criterion.to(device)

MSELoss()

In [47]:
device

device(type='cuda')

In [48]:
print(model)

Energy_Model(
  (pos_embedding): Embedding(25, 256)
  (num_embedding): Embedding(61, 256)
  (temp_embedding): Embedding(365, 256)
  (wind_spd_embedding): Embedding(203, 256)
  (humid_embedding): Embedding(1002, 256)
  (precip_embedding): Embedding(817, 256)
  (sun_embedding): Embedding(12, 256)
  (non_elec_embedding): Embedding(3, 256)
  (sunlight_embedding): Embedding(3, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
 

### Model_training

In [49]:
def train_epoch(model, train_dataloader, val_dataloader, optimizer, criterion, device="cuda"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    energies = []
    outs = []

    start_time = time.time()

    ## training
    for item in train_dataloader:
        num = item[0].to(device).long()
        temp = item[2].to(device).long()
        wind_spd = item[3].to(device).long()
        humid = item[4].to(device).long()
        precip = item[5].to(device).long()
        sun = item[6].to(device).long()
        non_elec = item[7].to(device).long()
        sunlight = item[8].to(device).long()

        # Target
        energy = item[1].to(device).float()
        
        target_mask = (num != 0)

        optimizer.zero_grad()
        output = model(num, temp, wind_spd, humid, precip, sun, non_elec, sunlight)
        
        loss = criterion(output, energy)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        
        
        # mask the output
        output_mask = torch.masked_select(output, target_mask)
        energy_mask = torch.masked_select(energy, target_mask)


        energies.extend(energy_mask.view(-1).data.cpu().numpy())
        outs.extend(output_mask.view(-1).data.cpu().numpy())

#     train_auc = roc_auc_score(energies, outs)
    train_loss = np.mean(train_loss)

    energies = []
    outs = []
    val_loss = []

    
    # validation
    model.eval()
    for item in val_dataloader:
        num = item[0].to(device).long()
        temp = item[2].to(device).long()
        wind_spd = item[3].to(device).long()
        humid = item[4].to(device).long()
        precip = item[5].to(device).long()
        sun = item[6].to(device).long()
        non_elec = item[7].to(device).long()
        sunlight = item[8].to(device).long()
        
        # Target
        energy = item[1].to(device).float()
        
        target_mask = (num != 0)
        
        output = model(num, temp, wind_spd, humid, precip, sun, non_elec, sunlight)

        
        ## mask the output
        output = torch.masked_select(output, target_mask)
        energy = torch.masked_select(energy, target_mask)
        
        
        loss = criterion(output, energy)
        val_loss.append(loss.item())

        energies.extend(energy.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

#     val_auc = roc_auc_score(energies, outs)
    val_loss = np.mean(val_loss)

    elapsed_time = time.time() - start_time 

#     return train_loss, train_auc, val_loss, val_auc, elapsed_time
    return train_loss, val_loss, elapsed_time

In [ ]:
epochs = 10
for epoch in range(epochs):
#     train_loss, train_auc, val_loss, val_auc, elapsed_time = train_epoch(model, train_loader, val_loader, optimizer, criterion, device)
    train_loss, val_loss, elapsed_time = train_epoch(model, train_loader, val_loader, optimizer, criterion, device)
    
#     print("epoch - {} train_loss - {:.4f} train_auc - {:.4f} val_loss - {:.4f} val_auc - {:.4f} time={:.2f}s".format(epoch, train_loss, train_auc, val_loss, val_auc, elapsed_time))
    print("epoch - {}  train_loss - {:.4f}  val_loss - {:.4f}  time={:.2f}s".format(epoch, train_loss, val_loss, elapsed_time))
#     logging.info("epoch - {} train_loss - {:.4f} train_auc - {:.4f} val_loss - {:.4f} val_auc - {:.4f} time={:.2f}s".format(epoch, train_loss, train_auc, val_loss, val_auc, elapsed_time))
    logging.info("epoch - {} train_loss - {:.4f}  val_loss - {:.4f}  time={:.2f}s".format(epoch, train_loss, val_loss, elapsed_time))

In [51]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
#             correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res